# Figure 5

## Imports

In [ ]:
import dolfin
import numpy    as np
import sympy    as sp
import matplotlib.pyplot as plt
from sympy.plotting import plot
import math
import pickle
from numpy import linspace
from sympy import lambdify
from mpl_toolkits.mplot3d import Axes3D
import os
import shutil

import myPythonLibrary as mypy
import dolfin_mech     as dmech
from numpy import exp,arange
from pylab import meshgrid,cm,imshow,contour,clabel,colorbar,axis,title,show

from scipy.optimize import minimize
from scipy.optimize import fsolve

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

import seeds
import geometry

In [ ]:
directory = "Hexagon"
parent_dir = "./Geometries"
path = os.path.join(parent_dir, directory) 
if os.path.exists(path):
    shutil.rmtree(path)
os.makedirs(path)

directory = "Hallow_square"
parent_dir = "./Geometries"
path = os.path.join(parent_dir, directory) 
if os.path.exists(path):
    shutil.rmtree(path)
os.makedirs(path)


## Parameters

In [ ]:
E_s  = 1
nu_s = 0.2

mu_s    = E_s/2/(1 + nu_s)
lmbda_s = E_s*nu_s/(1 + nu_s)/(1 - 2*nu_s)
k_s     = 3*lmbda_s + 2*mu_s

mat_params = {"model":"CGNH", "parameters":{"E":E_s, "nu":nu_s}}

# Random Geometry

In [ ]:
lmbda_lst_random = []
mu_lst_random    = []
phi_lst_random   = []
kappa_lst_random = []

fname = "Geometries/Hexagon/hexagon_RVE1_voronoi"

domain = 1
row = 3
DoI = 0.4
shift_y = 0
# seeds.semi_regular(DoI, row, domain)

thickness_lst = [0.01, 0.04, 0.06, 0.09, 0.12, 0.15, 0.17]
for i in range(len(thickness_lst)):
    print(thickness_lst[i])
    thickness = thickness_lst[i]
    geometry.voronoi_tessellation(fname, thickness, row, domain, shift_y, seeds_remove = None)
    mesh = dolfin.Mesh()
    dolfin.XDMFFile(fname+"-mesh.xdmf").read(mesh)

    coord = mesh.coordinates()
    xmax = max(coord[:,0]); xmin = min(coord[:,0])
    ymax = max(coord[:,1]); ymin = min(coord[:,1])
    vol = (xmax - xmin)*(ymax - ymin)


    bbox = [xmin, xmax, ymin, ymax]
    

    dV = dolfin.Measure(
        "dx",
        domain=mesh)
    vs0 = dolfin.assemble(dolfin.Constant(1) * dV)


    homog = dmech.HomogenizationProblem(dim=2,
                                mesh=mesh,
                                mat_params=mat_params["parameters"],
                                vol=vol,
                                bbox=bbox)
    [lambda_, mu_] = homog.get_lambda_and_mu()
    kappa_ = homog.get_kappa()

    phi_lst_random.append((vol - vs0)/vol)
    lmbda_lst_random.append(lambda_)
    mu_lst_random.append(mu_)
    kappa_lst_random.append(kappa_)


# Hexagon

In [ ]:
%%capture
lmbda_lst_hexagon =[]
mu_lst_hexagon = []
phi_lst_hexagon = []
kappa_lst_hexagon = []

fname = "Geometries/Hexagon/hexagon_RVE1"

domain = 1
row = 1
DoI = 0.0
shift_y = 0
seeds.semi_regular(DoI, row, domain)

for h in range(11):
    thickness = (h+0.5)/20
    geometry.voronoi(fname, thickness, row, domain, shift_y, seeds_remove = None)
    mesh = dolfin.Mesh()
    dolfin.XDMFFile(fname+"-mesh.xdmf").read(mesh)

    coord = mesh.coordinates()
    xmax = max(coord[:,0]); xmin = min(coord[:,0])
    ymax = max(coord[:,1]); ymin = min(coord[:,1])
    vol = (xmax - xmin)*(ymax - ymin)

    bbox = [xmin, xmax, ymin, ymax]
    

    dV = dolfin.Measure(
        "dx",
        domain=mesh)
    vs0 = dolfin.assemble(dolfin.Constant(1) * dV)


    homog = dmech.HomogenizationProblem(dim=2,
                                mesh=mesh,
                                mat_params=mat_params["parameters"],
                                vol=vol,
                                bbox=bbox)
    [lambda_, mu_] = homog.get_lambda_and_mu()
    kappa_ = homog.get_kappa()

    phi_lst_hexagon.append((vol - vs0)/vol)
    lmbda_lst_hexagon.append(lambda_)
    mu_lst_hexagon.append(mu_)
    kappa_lst_hexagon.append(kappa_)


# Circular inclusion

In [ ]:
%%capture
lmbda_lst_rectangle =[]
mu_lst_rectangle = []
phi_lst_rectangle = []
kappa_lst_rectangle = []


width = 1

vol = math.sqrt(3)*width**2



shift_x = 0
shift_y = 0

fname = 'Geometries/Hallow_square/Hex_rectangle'


for phi in range(1,85,2):
    porosity = phi/100
    r = math.sqrt(porosity*vol/math.pi/2)
    geometry.Hex_rectangle(fname, width, r, shift_x, shift_y)
    mesh = dolfin.Mesh()
    dolfin.XDMFFile(fname+"-mesh.xdmf").read(mesh)
    

    coord = mesh.coordinates()
    xmax = max(coord[:,0]); xmin = min(coord[:,0])
    ymax = max(coord[:,1]); ymin = min(coord[:,1])
    vol = (xmax - xmin)*(ymax - ymin)

    bbox = [xmin, xmax, ymin, ymax]
    

    dV = dolfin.Measure(
        "dx",
        domain=mesh)
    vs0 = dolfin.assemble(dolfin.Constant(1) * dV)


    homog = dmech.HomogenizationProblem(dim=2,
                                mesh=mesh,
                                mat_params=mat_params["parameters"],
                                vol=vol,
                                bbox=bbox)
    [lambda_, mu_] = homog.get_lambda_and_mu()
    kappa_ = homog.get_kappa()

    phi_lst_rectangle.append((vol - vs0)/vol)
    lmbda_lst_rectangle.append(lambda_)
    mu_lst_rectangle.append(mu_)
    kappa_lst_rectangle.append(kappa_)


In [ ]:
phi_dil = np.linspace(min(phi_lst_hexagon),4*mu_s/(4*mu_s + 3*k_s),100)
k_dil_norm = (1 - phi_dil*(1 + 3*k_s/4/mu_s))
plt.figure()

plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('legend', fontsize=12)
plt.xlabel(r'$\Phi_{f0}$', fontsize = 16)
plt.ylabel(r'$\tilde{\kappa}~(kPa)$', fontsize = 16)
# plt.plot(eps_glob, sigma_glob, 'r')
plt.plot(phi_lst_rectangle, kappa_lst_rectangle, '#084594')
plt.plot(phi_lst_hexagon, kappa_lst_hexagon, '#084594', linestyle='dashdot')
plt.plot(phi_lst_random, kappa_lst_random, '#084594', linestyle='dashed')


plt.xlim(0, 0.9)
plt.ylim(0, 25)
plt.legend([ 'Microscopic model - circular inclusions', 'Microscopic model - hexagonal inclusions', 'Microscopic model - polygonal inclusions'])


# plt.savefig('homogenized-bulk-modulus.pdf',bbox_inches='tight')
plt.show()

In [ ]:
k_lst_random = []
k_lst_hexagon = []
k_lst_rectangle = []

phi_lst_random.insert(0, 1.0)
mu_lst_random.insert(0, 0.0)
lmbda_lst_random.insert(0, 0.0)
kappa_lst_random.insert(0, 0.0)
phi_lst_random.append(0)
mu_lst_random.append(mu_s)
lmbda_lst_random.append(lmbda_s)
kappa_lst_random.append(k_s)
for i in range (len(phi_lst_random)):
    k_lst_random.append(3*lmbda_lst_random[i] + 2*mu_lst_random[i])

phi_lst_hexagon.insert(0, 1.0)
mu_lst_hexagon.insert(0, 0.0)
lmbda_lst_hexagon.insert(0, 0.0)
kappa_lst_hexagon.insert(0, 0.0)
phi_lst_hexagon.append(0)
mu_lst_hexagon.append(mu_s)
lmbda_lst_hexagon.append(lmbda_s)
kappa_lst_hexagon.append(k_s)
for i in range (len(phi_lst_hexagon)):
    k_lst_hexagon.append(3*lmbda_lst_hexagon[i] + 2*mu_lst_hexagon[i])

phi_lst_rectangle.insert(0, 0.)
phi_lst_rectangle.append(1.0)
mu_lst_rectangle.insert(0, mu_s)
lmbda_lst_rectangle.insert(0, lmbda_s)
kappa_lst_rectangle.insert(0, k_s)
mu_lst_rectangle.append(0.)
lmbda_lst_rectangle.append(0.)
kappa_lst_rectangle.append(0)
for i in range (len(phi_lst_rectangle)):
    k_lst_rectangle.append(3*lmbda_lst_rectangle[i] + 2*mu_lst_rectangle[i])



### Obtaining homogenized parameters from Dilute and Differential scheme by Dormieux Microporomechanics

In [ ]:
def Eq1(mu_hom, phi):
    return ((1+4*mu_s/(3*k_s))*(mu_hom/mu_s)**3)/(2 - (1 - 4*mu_s/(3*k_s))*(mu_hom/mu_s)**(3/5)) - (1 - phi)**6

def Eq2(mu_hom, k_hom):
    return mu_hom/mu_s - ((-4/3*mu_hom/k_hom + 1)**(5/3))/((-4/3*mu_s/k_s + 1)**(5/3))

def Differential_scheme(phi_0):
    h = 10e-6
    mu_hom = mu_s
    k_hom = k_s
    res = 1
    while (res > 10e-7):
        res = Eq1(mu_hom, phi_0)
        jac = (Eq1(mu_hom + h, phi_0) - Eq1(mu_hom, phi_0))/h
        delta = - res/jac
        mu_hom = mu_hom + delta

    res = 1
    while (res > 10e-7):
        res = Eq2(mu_hom, k_hom)
        jac = (Eq2(mu_hom, k_hom + h) - Eq2(mu_hom, k_hom))/h
        delta = - res/jac
        k_hom = k_hom + delta

    return mu_hom, k_hom

def k_diff_func(mu_hom):
    return 4/3*mu_hom/(1 - (mu_hom/mu_s)**(3/5)*(1 - 4/3*mu_s/k_s))

In [ ]:
mu_diff = []
k_diff = []
mu_dil = []
k_dil = []
for i in range(len(phi_lst_hexagon)):
    mu_ = Differential_scheme(phi_lst_hexagon[i])[0]
    mu_diff.append(mu_)
    k_diff.append(k_diff_func(mu_))
    mu_dil.append(mu_s*(1 - 5*phi_lst_hexagon[i]*(3*k_s + 4*mu_s)/(9*k_s + 8*mu_s)))
    k_dil.append(k_s*(1 - phi_lst_hexagon[i]*(1 + 3*k_s/4/mu_s)))


## Normalizing the homogenized parameters

In [ ]:
mu_lst_hexagon_norm = []
kappa_lst_hexagon_norm = []
mu_diff_norm = []
mu_dil_norm = []
k_lst_hexagon_norm = []
k_diff_norm = []
k_dil_norm = []

mu_lst_norm_random = []
k_lst_norm_random = []
kappa_lst_norm_random = []

mu_lst_norm_rectangle = []
k_lst_norm_rectangle = []
kappa_lst_norm_rectangle = []

for i in range (len(mu_lst_hexagon)):
    mu_lst_hexagon_norm.append(mu_lst_hexagon[i]/mu_s)
    mu_dil_norm.append(mu_dil[i]/mu_s)
    mu_diff_norm.append(mu_diff[i]/mu_s)
    k_lst_hexagon_norm.append(k_lst_hexagon[i]/k_s)
    k_diff_norm.append(k_diff[i]/k_s)
    k_dil_norm.append(k_dil[i]/k_s)
    kappa_lst_hexagon_norm.append(kappa_lst_hexagon[i]/k_s)


for i in range (len(mu_lst_random)):
    mu_lst_norm_random.append(mu_lst_random[i]/mu_s)
    k_lst_norm_random.append(k_lst_random[i]/k_s)
    kappa_lst_norm_random.append(kappa_lst_random[i]/k_s)


for i in range (len(mu_lst_rectangle)):
    mu_lst_norm_rectangle.append(mu_lst_rectangle[i]/mu_s)
    k_lst_norm_rectangle.append(k_lst_rectangle[i]/k_s)
    kappa_lst_norm_rectangle.append(kappa_lst_rectangle[i]/k_s)

In [ ]:
phi_dil = np.linspace(min(phi_lst_hexagon),(9*k_s + 8*mu_s)/(3*k_s + 4*mu_s)/5,100)
mu_dil_norm = 1 - 5*phi_dil*(3*k_s + 4*mu_s)/(9*k_s + 8*mu_s)
plt.figure()
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('legend', fontsize=12)
plt.xlabel(r'$\Phi_{f0}$', fontsize=16)
plt.ylabel(r'$\tilde{\mu}/\mu_s$', fontsize=16)
# plt.plot(eps_glob, sigma_glob, 'r')
plt.plot(phi_lst_rectangle, mu_lst_norm_rectangle, '#084594')
plt.plot(phi_lst_hexagon, mu_lst_hexagon_norm, '#084594', linestyle='dashdot')
plt.plot(phi_lst_random, mu_lst_norm_random, '#084594', linestyle='dashed')
plt.plot(phi_lst_hexagon, mu_diff_norm, '#005824')
plt.plot(phi_dil, mu_dil_norm, '#005824', linestyle='dashed')
# plt.plot(phi_lst[2:], mu_dor_lin_norm[2:], '#E31A1C')
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.legend(['Microscopic model - circular inclusions', 'Microscopic model - hexagonal inclusions', 'Microscopic model - polygonal inclusions', 'Differential scheme [Dormieux et al. 2006]', 'Dilute scheme [Dormieux et al. 2006]'])
# plt.scatter(phi_lst_hexagon, mu_lst_hexagon_norm)
plt.text(phi_lst_hexagon[9]-0.02, mu_lst_hexagon_norm[9]-0.08, '(a)',fontsize=12)
plt.text(phi_lst_hexagon[6]-0.015, mu_lst_hexagon_norm[6]-0.07, '(b)',fontsize=12)
plt.text(phi_lst_hexagon[4]-0.025, mu_lst_hexagon_norm[4]-0.04, '(c)',fontsize=12)
plt.text(phi_lst_hexagon[2]-0.015, mu_lst_hexagon_norm[2]+0.02, '(d)',fontsize=12)

# plt.savefig('linear_Dormieux_mu.pdf',bbox_inches='tight')
plt.show()

In [ ]:
phi_dil = np.linspace(min(phi_lst_hexagon),4*mu_s/(4*mu_s + 3*k_s),100)
k_dil_norm = (1 - phi_dil*(1 + 3*k_s/4/mu_s))
plt.figure()

plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('legend', fontsize=12)
plt.xlabel(r'$\Phi_{f0}$', fontsize = 16)
plt.ylabel(r'$\tilde{k}/k_s$', fontsize = 16)
# plt.plot(eps_glob, sigma_glob, 'r')
plt.plot(phi_lst_rectangle, k_lst_norm_rectangle, '#084594')
plt.plot(phi_lst_hexagon, k_lst_hexagon_norm, '#084594', linestyle='dashdot')
plt.plot(phi_lst_random, k_lst_norm_random, '#084594', linestyle='dashed')
plt.plot(phi_lst_hexagon, k_diff_norm, '#005824')
plt.plot(phi_dil, k_dil_norm, '#005824', linestyle='dashed')
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.legend(['Microscopic model - circular inclusions', 'Microscopic model - hexagonal inclusions', 'Microscopic model - polygonal inclusions', 'Differential scheme [Dormieux et al. 2006]', 'Dilute scheme [Dormieux et al. 2006]'])

# plt.scatter(phi_lst_hexagon, k_lst_hexagon_norm)
plt.text(phi_lst_hexagon[9]+0.01, k_lst_hexagon_norm[9], '(a)',fontsize=12)
plt.text(phi_lst_hexagon[6], k_lst_hexagon_norm[6]+0.025, '(b)',fontsize=12)
plt.text(phi_lst_hexagon[4]-0.015, k_lst_hexagon_norm[4]+0.025, '(c)',fontsize=12)
plt.text(phi_lst_hexagon[2]-0.015, k_lst_hexagon_norm[2]+0.03, '(d)',fontsize=12)
# plt.savefig('linear_Dormieux_k.pdf',bbox_inches='tight')
plt.show()
